In [1]:
from podpac.core.coordinates import Coordinates, clinspace
from podpac.core.data import interpolate
from podpac.core.data.interpolate import (Interpolation, InterpolationException,
                                          Interpolator, INTERPOLATION_METHODS,
                                          INTERPOLATION_SHORTCUTS, NearestNeighbor)

In [2]:
COORDINATES = Coordinates([clinspace(-25, 25, 11), clinspace(-25, 25, 11)], dims=['lat', 'lon'])
STACKED_COORDINATES = Coordinates([([1,2,3,4,5], [0,1,2,3,4])], dims=['lat_lon'])

In [3]:
interp = Interpolation('nearest', COORDINATES)

In [4]:
interp

In [11]:
isinstance(interp._definition['lat'][1][0], Interpolator)

True